# Generation of BoM Data for Supply Chain Simulation

In [ ]:
SAVE_RECORDS=True

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import random
import pandas as pd
import json

## Mock Supply Chain

In [ ]:
from core.supply_chain import SupplyChain, LeafNode, CombinerNode, SinkNode

def create_supply_chain():
    supply_chain = SupplyChain()

    # Create leaf nodes with random cost types
    leaf1 = LeafNode(200, "raw_material_1", [(0, 10), (5, 15), (10, 20)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf2 = LeafNode(250, "raw_material_2", [(0, 15), (5, 20), (10, 25)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf3 = LeafNode(220, "raw_material_3", [(0, 12), (5, 18), (10, 22)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(leaf1)
    supply_chain.add_node(leaf2)
    supply_chain.add_node(leaf3)

    # Create intermediate nodes with random cost types
    intermediate_a1 = CombinerNode(300, "intermediate_a1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_a2 = CombinerNode(280, "intermediate_a2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_a1)
    supply_chain.add_node(intermediate_a2)

    intermediate_b1 = CombinerNode(300, "intermediate_b1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_b2 = CombinerNode(280, "intermediate_b2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_b1)
    supply_chain.add_node(intermediate_b2)

    # Create final combiner node with random cost type
    final_combiner = CombinerNode(350, "final_product", [0.7, 0.8, 0.9], [1, 1.1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(final_combiner)

    # Create sink node with random cost type
    sink = SinkNode(consumption_rate=250, cost_type=random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(sink)

    # Add edges with cost ranges
    supply_chain.add_edge(leaf1, intermediate_a1, 10, 30, 5, 15)
    supply_chain.add_edge(leaf2, intermediate_a1, 15, 35, 8, 20)
    supply_chain.add_edge(leaf2, intermediate_a2, 12, 30, 6, 18)
    supply_chain.add_edge(leaf3, intermediate_a2, 14, 32, 7, 19)
    supply_chain.add_edge(intermediate_a1, intermediate_b1, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_a2, intermediate_b1, 22, 42, 11, 27)
    supply_chain.add_edge(intermediate_b1, final_combiner, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_b2, final_combiner, 22, 42, 11, 27)
    supply_chain.add_edge(leaf3, final_combiner, 18, 35, 9, 23)
    supply_chain.add_edge(final_combiner, sink, 25, 50, 13, 30)

    return supply_chain

In [ ]:
# Simulation
time = 0
supply_chain = create_supply_chain()

for _ in range(200):  # Simulate for 200 time steps
    supply_chain.update()
    time += 1

    # Print current state
    print(f"Time: {time}")
    for node in supply_chain.nodes:
        print(f"Node ({node.node_class}): Inventory = {node.inventory}, Last Production = {node.last_production}, Cost Type = {node.cost_type}")
        for edge, target_node in node.outgoing_edges:
            print(f"  Edge to {target_node.node_class}: Quantity = {edge.quantity}, Cost = {edge.current_cost:.2f}")
    print("---")

## Recording of the simulation

In [ ]:
def simulate_and_collect_data(supply_chain, n_cycles):
    # Initialize DataFrames
    metadata = []
    node_data = []
    edge_data = []
    
    node_id = 0

    # Create metadata and assign unique IDs
    for node in supply_chain.nodes:
        metadata.append({
            'node_id': f"node_{node_id}",
            'node_class': node.node_class,
            'max_inventory': node.max_inventory,
            'cost_type': node.cost_type
        })
        node.id = node_id  # Assign ID to node object for reference
        node_id += 1

    edge_id = 0
    # Create edge metadata
    for edge in supply_chain.edges:
        source_node = None
        target_node = None
        
        for node in supply_chain.nodes:
            if edge in [e for e, _ in node.outgoing_edges]:
                source_node = node
            if edge in [e for e, _ in node.incoming_edges]:
                target_node = node
            if source_node and target_node:
                break
        
        if not source_node or not target_node:
            print(f"Warning: Edge {edge_id} is not properly connected.")
            print(f"Source node: {source_node.node_class if source_node else 'None'}")
            print(f"Target node: {target_node.node_class if target_node else 'None'}")
            continue
        
        metadata.append({
            'edge_id': f"edge_{edge_id}",
            'source_node_id': source_node.id,
            'target_node_id': target_node.id,
            'unit_price': edge.unit_price,
            'min_cost': edge.min_cost,
            'max_cost': edge.max_cost
        })
        edge.id = edge_id  # Assign ID to edge object for reference
        edge_id += 1

    # Simulate for n cycles
    for cycle in range(n_cycles):
        supply_chain.update()

        # Collect node data
        for node in supply_chain.nodes:
            node_data.append({
                'cycle': cycle,
                'node_id': f"node_{node.id}",
                'inventory': node.inventory,
                'last_production': node.last_production
            })

        # Collect edge data
        for edge in supply_chain.edges:
            if hasattr(edge, 'id'):  # Only collect data for edges that were properly connected
                edge_data.append({
                    'cycle': cycle,
                    'edge_id': f"edge_{edge.id}",
                    'quantity': edge.quantity,
                    'current_cost': edge.current_cost
                })

    # Create DataFrames
    metadata_df = pd.DataFrame(metadata)
    node_data_df = pd.DataFrame(node_data)
    edge_data_df = pd.DataFrame(edge_data)

    return metadata_df, node_data_df, edge_data_df

In [ ]:
supply_chain = create_supply_chain()
metadata_df, node_data_df, edge_data_df = simulate_and_collect_data(supply_chain, n_cycles=200)

In [ ]:
metadata_df

In [ ]:
node_data_df

In [ ]:
edge_data_df

## Conversion between JSON and DataFrames

In [ ]:
from typing import Dict, List

def dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Process node data
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        bom["nodes"][node_id][cycle] = row.to_dict()

    # Process edge data
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        bom["edges"][edge_id][cycle] = row.to_dict()

    return json.dumps(bom, indent=2)

def json_to_dataframes(json_data: str) -> Dict[str, pd.DataFrame]:
    bom = json.loads(json_data)

    # Create empty lists to store the data
    node_data = []
    edge_data = []

    # Process nodes
    for node_id, cycles in bom["nodes"].items():
        for cycle, node_info in cycles.items():
            node_data.append(node_info)

    # Process edges
    for edge_id, cycles in bom["edges"].items():
        for cycle, edge_info in cycles.items():
            edge_data.append(edge_info)

    # Create DataFrames
    metadata_df = pd.DataFrame(bom["metadata"])
    node_df = pd.DataFrame(node_data)
    edge_df = pd.DataFrame(edge_data)

    return {
        "metadata": metadata_df,
        "node": node_df,
        "edge": edge_df
    }

In [ ]:
# Convert dataframes to JSON
json_string = dataframes_to_json(metadata_df, node_data_df, edge_data_df)

In [ ]:
json_data = json.loads(json_string)
json_data

In [ ]:
if SAVE_RECORDS:
    with open('../data/json/true.json', 'w') as f:
        json.dump(json_data, f, ensure_ascii=False)

In [ ]:
# Convert JSON back to dataframes
reconstructed_dfs = json_to_dataframes(json_string)

# Access the reconstructed dataframes
reconstructed_metadata = reconstructed_dfs['metadata']
reconstructed_node = reconstructed_dfs['node']
reconstructed_edge = reconstructed_dfs['edge']

In [ ]:
if SAVE_RECORDS:
    reconstructed_metadata.to_csv('../data/csv/metadata.csv', index=False)
    reconstructed_node.to_csv('../data/csv/node.csv', index=False)
    reconstructed_edge.to_csv('../data/csv/edge.csv', index=False)

In [ ]:
reconstructed_metadata

In [ ]:
reconstructed_node

In [ ]:
reconstructed_edge 

## Aggredation of Cycles and Conversion between JSON and Dataframes

In [ ]:
from typing import Dict

def aggregated_dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame, max_aggregation_cycles: int = 3) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Initialize variables to track the last cycle
    last_node_cycle = 0
    last_edge_cycle = 0
    
    num_cycles_to_aggregate = random.randint(1, max_aggregation_cycles)

    # Process node data with aggregation
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        
        # Aggregate nodes
        for _ in range(num_cycles_to_aggregate):
            last_node_cycle += 1
            # Create a new entry for the aggregated node
            aggregated_node = row.copy()
            aggregated_node['cycle'] = last_node_cycle
            bom["nodes"][node_id][last_node_cycle] = aggregated_node.to_dict()

    # Process edge data with aggregation
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        
        # Aggregate edges
        for _ in range(num_cycles_to_aggregate):
            last_edge_cycle += 1
            # Create a new entry for the aggregated edge
            aggregated_edge = row.copy()
            aggregated_edge['cycle'] = last_edge_cycle
            bom["edges"][edge_id][last_edge_cycle] = aggregated_edge.to_dict()

    return json.dumps(bom, indent=2)

In [ ]:
aggregated_json = aggregated_dataframes_to_json(metadata_df, node_data_df, edge_data_df)
aggregated_json_dict = json.loads(aggregated_json)

aggregated_json_dict

In [ ]:
if SAVE_RECORDS:
    import json
    with open('../data/json/aggregated.json', 'w') as f:
        json.dump(aggregated_json_dict, f, ensure_ascii=False)

In [ ]:
aggregated_dataframe = json_to_dataframes(aggregated_json)

aggregated_metadata = aggregated_dataframe["metadata"]
aggregated_nodes = aggregated_dataframe["node"]
aggregated_edges = aggregated_dataframe["edge"]

In [ ]:
if SAVE_RECORDS:
    aggregated_metadata.to_csv("../data/csv/aggregated_metadata.csv", index=False)
    aggregated_nodes.to_csv("../data/csv/aggregated_node_data.csv", index=False)
    aggregated_edges.to_csv("../data/csv/aggregated_edge_data.csv", index=False)

In [ ]:
aggregated_metadata

In [ ]:
aggregated_nodes

In [ ]:
aggregated_edges